In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
from pandas import DataFrame as df
import math as m
%matplotlib qt

In [ ]:
##Questa sezione importa i dati (sono state già selezionate le colonne, per ridurre la dimensione del file) e li ordina per
###nome degli oggetti e velocità radiali
data=pd.read_csv('./Settings/Data/Default/data_select.csv')
data=data.sort_values(by=['GES_FLD','VRAD'])
data=data.reset_index(drop=True)

In [ ]:
###Questa sezione conteggia il numero di ammassi nel campione; n ha gli indici in cui compare per la prima volta un nuovo ammasso
###(nb: sono stati ordinati per il nome dell'ammasso), invece obj è un array con i nomi degli ammassi. In output restituisce due
###array, uno che consiste negli indici numerici dei nuovi ammassi mentre l'altro è la lista dei nomi dei diversi ammassi.
n=[0]
for i in range(1,data.shape[0]):
    if data.at[i,'GES_FLD']==data.at[i-1,'GES_FLD']:
        pass
    else:
        n.append(i)
#print(n)
obj=[]
for i in range(0,len(n)):
    obj.append(data.at[n[i],'GES_FLD'])
print(obj)
l=len(obj)
n.append(1418)

In [ ]:
###Questa sezione calcola la velocità radiale media; per farlo, prima scarta i casi in cui non è stata calcolata la velocità,
###dopo di che fa una media pesata sugli errori relativi a VRAD. Segue una selezione tramite STDDEV, ricalcolando quindi la 
###velocità media dopo aver scartato gli outlier. In output da tre array, il primo che è la lista delle velocità medie per ogni
###ammasso, il secondo è la lista dei nomi delle stelle escluse dalla selezione per STDDEV, infine il terzo è la lista degli 
###indici delle stelle escluse.
med_v=[]
velex=[]
velex_in=[]
for j in range(0,l):
    a=[]
    e=[]
    ind=[]
    for i in range(n[j],n[j+1]):
        if m.isnan(data.at[i,'VRAD'])==True:
            velex.append(data.at[i,'CNAME'])
            velex_in.append(i)
        else:
            a.append(data.at[i,'VRAD'])
            e.append(data.at[i,'E_VRAD'])
            ind.append(i)
    v=np.average(a,weights=e)
    stddev=np.std(a)
    #var=np.average((a-v)**2,weights=e)
    #ptile=np.percentile(a,4.5)
    #ptile2=np.percentile(a,95.5)
    na=[]
    ne=[]
    for k in range(0,len(a)):
        if a[k]>=(v-1*stddev) and a[k]<=(v+1*stddev):
            na.append(a[k])
            ne.append(e[k])
        else:
            velex.append(data.at[ind[k],'CNAME'])
            velex_in.append(ind[k])
    defv=np.average(na,weights=ne)
    med_v.append(defv)

In [ ]:
###Questa sezione si occupa del plot delle velocità radiali suddivise per ammasso. In grigio, i punti scartati dalla selezione;
###in blu, i punti validi dalla selezione; la barra rossa indica la velocità radiale media riferita al singolo oggetto.
plt.close()
plt.grid(axis='y',linestyle=':')
plt.axvline(x=0,color='grey',linestyle='--',linewidth=0.7)
for i in range(0,data.shape[0]):
    if data.at[i,'CNAME'] in velex:
        plt.scatter(data.at[i,'VRAD'],data.at[i,'GES_FLD'],marker='.',color='grey')
        
    else:
        plt.scatter(data.at[i,'VRAD'],data.at[i,'GES_FLD'],marker='.',color='blue')
plt.scatter(med_v,obj,marker='|',color='red')
plt.xlabel('VRAD (km/s)')

In [ ]:
plt.close()
plt.grid(axis='y',linestyle=':')
plt.axvline(x=0,color='grey',linestyle='--',linewidth=0.7)
for i in range(0,data.shape[0]):
    if data.at[i,'CNAME'] in velex:
        pass   
    else:
        plt.scatter(data.at[i,'VRAD'],data.at[i,'GES_FLD'],marker='.',color='blue')
plt.scatter(med_v,obj,marker='|',color='red')

In [ ]:
###Si creano gli array relativi alle varie isocrone; isom sarà quella più giovane, isoM quella più vecchia, la terza "a piacere".
isom=pd.read_csv('./Settings/Isophotes/Default/iso60000.csv',sep=';')
isoM=pd.read_csv('./Settings/Isophotes/Default/iso98451.csv',sep=';')
isoa=pd.read_csv('./Settings/Isophotes/Default/iso70000.csv',sep=';')
isob=pd.read_csv('./Settings/Isophotes/Default/iso80000.csv',sep=';')
isoc=pd.read_csv('./Settings/Isophotes/Default/iso90000.csv',sep=';')
isomt=[]
isomg=[]
isoMt=[]
isoMg=[]
isoat=[]
isoag=[]
isobt=[]
isobg=[]
isoct=[]
isocg=[]
for i in range(0,isom.shape[0]):
    isomt.append(isom.at[i,'logTe'])
    isomg.append(isom.at[i,'logg'])
for i in range(0,isoM.shape[0]):
    isoMt.append(isoM.at[i,'logTe'])
    isoMg.append(isoM.at[i,'logg'])
for i in range(0,isoa.shape[0]):
    isoat.append(isoa.at[i,'logTe'])
    isoag.append(isoa.at[i,'logg'])
for i in range(0,isob.shape[0]):
    isobt.append(isob.at[i,'logTe'])
    isobg.append(isob.at[i,'logg'])
for i in range(0,isoc.shape[0]):
    isoct.append(isoc.at[i,'logTe'])
    isocg.append(isoc.at[i,'logg'])

In [ ]:
###In questa sezione si inizia l'analisi sul CMD: creando due array (serviranno poi per HDBSCAN) con Teff e Logg;
###si crea qui un altro array, contenente gli indici delle diverse stelle. Alla fine, si uniscono i due arrays in
###un unico array con due colonne, utile al plot.
N=[]
Teff=[]
Logg=[]
for i in range(0,data.shape[0]):
    N.append(i)
    Teff.append(np.log10(data.at[i,'TEFF']))
    Logg.append(data.at[i,'LOGG'])

b=np.column_stack([Teff,Logg])

In [ ]:
###Questa sezione fa il plot del grafico CMD, espresso con Log(g) su Log(Teff). In grigio i punti esclusi in precedenza,
###in blu gli altri.
plt.close()
c1=0
c2=0
for i in range(0,data.shape[0]):
    if i in velex_in:
        if c1==0:
            plt.scatter(Teff[i],Logg[i],marker='.',color='grey',facecolors='none',label='Rejected stars')
            c1=1
        else:
            plt.scatter(Teff[i],Logg[i],marker='.',color='grey',facecolors='none')
    else:
        if c2==0:
            plt.scatter(Teff[i],Logg[i],marker='.',color= 'black',facecolors='none',label=' Selected stars')
            c2=1
        else:
            plt.scatter(Teff[i],Logg[i],marker='.',color= 'black',facecolors='none')
plt.plot(isomt,isomg,color='blue',linestyle='-',label='1.0 Myr')
plt.plot(isoat,isoag,color='green',linestyle='-',label='10.0 Myr')
plt.plot(isobt,isobg,color='magenta',linestyle='-',label='0.1 Gyr')
plt.plot(isoct,isocg,color='green',linestyle='--', label='1.0 Gyr')
plt.plot(isoMt,isoMg,color='blue',linestyle='--',label='7.0 Gyr')

ax=plt.gca()
ax.invert_xaxis()
plt.xlabel('Log(Teff)')
ax.invert_yaxis()
plt.ylabel('Log(G)')
plt.legend()

In [ ]:
###Si applica un taglio, in particolare si tagliano tutte le stelle con Log(Teff)>4.0 . Il motivo di questa scelta si lega alla
###natura di queste stelle: sono stelle molto brillanti (di classe O e B) appartenenti ad ammassi molto giovani. In queste stelle
###risulta quasi impossibile misurare efficaciemente le abbondanze chimiche. Questa selezione quindi rimuove una parte del
###campione che verrebbe comunque tolto per l'assenza di misurazioni nelle abbondanze chimiche.
veltex_in=[]
tex_in=[]
cut=4.0
for i in range(0,data.shape[0]):
    if (m.log10(data.at[i,'TEFF']))>cut:
        tex_in.append(i)
    else:
        pass

In [ ]:
###Vengono uniti gli array relativi alle due selezioni, ordinati e eliminate le ripetizioni al loro interno. Si definisce c,
###ossia il nuovo campione su cui si applica HDBScan.
veltex_in=np.concatenate((velex_in,tex_in),0)
veltex_in=np.sort(veltex_in)
veltex_in=np.unique(veltex_in)
c=np.delete(b,veltex_in,0)

In [ ]:
###In questa sezione si inizia a lavorare sui punti; si importa e si settano i parametri di HDBScan.
import hdbscan
clusterer=hdbscan.HDBSCAN(metric='l2',min_cluster_size=150,min_samples=10,cluster_selection_epsilon=10)
clusterer.fit(c)

In [ ]:
###Dopo aver applicato HDBScan, si fa qui una divisione tra oggetti inseriti in un cluster e oggetti non contenuti in alcun
###cluster dal comando. In questa sezione si dividono già gli indici tra MS e GB in due array, per creare poi i due file .csv.
n_clust=clusterer.labels_.max()
clex_in=[]
cTeff=[]
cLogg=[]
clabel=[]
MSind=[]
GBind=[]
i=0
for j in range(0,data.shape[0]):
    if j in veltex_in:
        pass
    else:
        if clusterer.labels_[i]==-1:
            clex_in.append(j)
            i+=1
        else:
            cTeff.append(m.log10(data.at[j,'TEFF']))
            cLogg.append(data.at[j,'LOGG'])
            clabel.append(clusterer.labels_[i])
            if clusterer.labels_[i]==0:
                GBind.append(j)
            elif clusterer.labels_[i]==1:
                MSind.append(j)
            i+=1

In [ ]:
###Si crea un array con gli indici di tutte le stelle escluse. Questa è l'ultima selezione prima di dividere il campione tra
###Main Sequence (MS) e Giant Branch (GB). Si creerà un altro file .csv per le stelle scartate dalle selezioni.
totex_in=np.concatenate((veltex_in,clex_in),0)
totex_in=np.sort(totex_in)

In [ ]:
###Si fa il plot della selezione di HDBScan.
plt.close()
c1=0
c2=0
for i in range(0,data.shape[0]):
    if i in veltex_in:
        if c1==0:
            plt.scatter(Teff[i],Logg[i],marker='.',color='grey',facecolors='none',label='Rejected stars')
            c1=1
        else:
            plt.scatter(Teff[i],Logg[i],marker='.',color='grey',facecolors='none')
    else:
        if c2==0:
            plt.scatter(Teff[i],Logg[i],marker='.',color= 'black',facecolors='none',label='Selected stars')
            c2=1
        else:
            plt.scatter(Teff[i],Logg[i],marker='.',color= 'black',facecolors='none')
plt.scatter(cTeff,cLogg,marker='.',color='red',label='HDBScan selected point')
plt.plot(isomt,isomg,color='blue',linestyle='-',label='1.0 Myr')
plt.plot(isoat,isoag,color='green',linestyle='-',label='10.0 Myr')
plt.plot(isobt,isobg,color='magenta',linestyle='-',label='0.1 Gyr')
plt.plot(isoct,isocg,color='green',linestyle='--', label='1.0 Gyr')
plt.plot(isoMt,isoMg,color='blue',linestyle='--',label='7.0 Gyr')
ax=plt.gca()
ax.invert_xaxis()
plt.xlabel('Log(Teff)')
ax.invert_yaxis()
plt.ylabel('Log(G)')
plt.legend()

In [ ]:
###Questo plot invece mostra esclusivamente il clustering, eliminando i dati scartati dalle varie selezioni inclusa
###quella fatta dal clustering stesso.
plt.close()
c1=0
c2=0
for i in range(0,len(cTeff)):
    if clabel[i]==0:
        if c1==0:
            plt.scatter(cTeff[i],cLogg[i],marker='.',color='red',facecolors='none',label='GB stars using HDBScan')
            c1=1
        else:
            plt.scatter(cTeff[i],cLogg[i],marker='.',color='red',facecolors='none')
    elif clabel[i]==1:
        if c2==0:
            plt.scatter(cTeff[i],cLogg[i],marker='.',color='black',facecolors='none',label='MS stars using HDBScan')
            c2=1
        else:
            plt.scatter(cTeff[i],cLogg[i],marker='.',color='black',facecolors='none')
plt.plot(isomt,isomg,color='blue',linestyle='-',label='1.0 Myr')
plt.plot(isoat,isoag,color='green',linestyle='-',label='10.0 Myr')
plt.plot(isobt,isobg,color='magenta',linestyle='-',label='0.1 Gyr')
plt.plot(isoct,isocg,color='green',linestyle='--', label='1.0 Gyr')
plt.plot(isoMt,isoMg,color='blue',linestyle='--',label='7.0 Gyr')
ax=plt.gca()
ax.invert_xaxis()
plt.xlabel('Log(Teff)')
ax.invert_yaxis()
plt.ylabel('Log(G)')
plt.legend()

In [ ]:
###In questa sezione si generano i file .csv con tutte le suddivisioni fatte.
exlist=data.loc[totex_in]
MSlist=data.loc[MSind]
GBlist=data.loc[GBind]
exlist.to_csv('./Settings/Data/ExclData.csv',index=False)
MSlist.to_csv('./Settings/Data/MSData.csv',index=False)
GBlist.to_csv('./Settings/Data/GBData.csv',index=False)

In [ ]:
###Da adesso, si ripete per tre volte lo stesso procedimento, fatto sulle diverse selezioni adottate. la spiegazione verrà
###fatta solo la prima volta, dopo di che ci si liimiterà a mostrare dove inizia il nuovo procedimento.

In [ ]:
###MSData
###Si procede adesso a selezionare gli elementi più presenti per numero di conteggi. La selezione può essere fatta
###tramite percentuale tramite il parametro perc (seleziona quegli elementi che compaiono almeno nel [perc]% del campione)
###oppure imponendo il numero di conteggi, scrivendo direttamente il numero desiderato nel parametro limc.
MSdata=pd.read_csv('./Settings/Data/Default/MSData.csv')
el=[]
for i in range(25,82):
    el.append(list(MSdata)[i])
eldata=MSdata[el]
c=eldata.count()
c.to_csv('./Settings/Data/MSelcount.csv')

count=pd.read_csv('./Settings/Data/MSelcount.csv',names=['Element','Count'])
goodel=[]
perc=70
limc=(perc)*MSdata.shape[0]/100
limms=limc

for i in range(25,82):
    if count.at[i-24,'Count']<limc:
        pass
    elif count.at[i-24,'Count']>=limc:
        goodel.append(i)
        
print(len(goodel),' elementi compaiono nel ',perc,'%  del campione (almeno ',int(limc),' conteggi).')
print(goodel)

In [ ]:
###Qui si fa il plot dei conteggi relativi ai vari elementi, per vedere quali sonop maggiormente significativi nel campione.
###NB: Non è detto che siano le stesse stelle ad avere tutti i conteggi più abbondanti, ma qui si stanno selezionando gli elementi.
plt.close()
count=count.sort_values(by=['Count'],ascending=False)
count=count.drop([0])
count=count.reset_index(drop=True)
#for i in range(0,len(count)):
for i in range(0,32):
    if count.at[i,'Count']>=limc:
        plt.barh(count.at[i,'Element'],width=count.at[i,'Count'],color='blue')
    else:
        plt.barh(count.at[i,'Element'],width=count.at[i,'Count'],color='grey')
plt.axvline(limc,color='red')
plt.title('Count of elements for MS Stars')
plt.xlabel('Counts')

In [ ]:
###Si selezionano gli indici relativi alle colonne che si vogliono mantenere.
mselected=[0,1,3,4,5,6,7,8,9,10,14,15]
selected=np.append(mselected,goodel)
selected=np.sort(selected)

In [ ]:
###Si crea infine il file con tutte le selezioni applicate.
headers=[]
i=0
for i in range(0,len(selected)):
    headers.append(list(MSdata)[selected[i]])

MSel=MSdata[headers]
MSel.to_csv('./Settings/Data/DEFMSData.csv', index=False)

In [ ]:
###GBData
GBdata=pd.read_csv('./Settings/Data/Default/GBData.csv')
el=[]
for i in range(25,82):
    el.append(list(GBdata)[i])
eldata=GBdata[el]
c=eldata.count()
c.to_csv('./Settings/Data/GBelcount.csv')

count=pd.read_csv('./Settings/Data/GBelcount.csv',names=['Element','Count'])
goodel=[]
perc=90
limc=(perc)*GBdata.shape[0]/100
limgb=limc

for i in range(25,82):
    if count.at[i-24,'Count']<limc:
        pass
    elif count.at[i-24,'Count']>=limc:
        goodel.append(i)
        
print(len(goodel),' elementi compaiono nel ',perc,'%  del campione (almeno ',int(limc),' conteggi).')
print(goodel)

In [ ]:
plt.close()
count=count.sort_values(by=['Count'],ascending=False)
count=count.drop([0])
count=count.reset_index(drop=True)
#for i in range(0,len(count)):
for i in range(0,31):
    if count.at[i,'Count']>=limc:
        plt.barh(count.at[i,'Element'],width=count.at[i,'Count'],color='blue')
    else:
        plt.barh(count.at[i,'Element'],width=count.at[i,'Count'],color='grey')
plt.axvline(limc,color='red')
plt.title('Count of elements for GB Stars')
plt.xlabel('Counts')

In [ ]:
mselected=[0,1,3,4,5,6,7,8,9,10,14,15]
selected=np.append(mselected,goodel)
selected=np.sort(selected)

In [ ]:
headers=[]
i=0
for i in range(0,len(selected)):
    headers.append(list(GBdata)[selected[i]])

GBel=GBdata[headers]
GBel.to_csv('./Settings/Data/DEFGBData.csv', index=False)

In [ ]:
###ExclData
Excldata=pd.read_csv('./Settings/Data/Default/ExclData.csv')
el=[]
for i in range(25,82):
    el.append(list(Excldata)[i])
eldata=Excldata[el]
c=eldata.count()
c.to_csv('./Settings/Data/Exclelcount.csv')

count=pd.read_csv('./Settings/Data/Exclelcount.csv',names=['Element','Count'])
goodel=[]
perc=40
limc=(perc)*Excldata.shape[0]/100
limex=limc

for i in range(25,82):
    if count.at[i-24,'Count']<limc:
        pass
    elif count.at[i-24,'Count']>=limc:
        goodel.append(i)
        
print(len(goodel),' elementi compaiono nel ',perc,'%  del campione (almeno ',int(limc),' conteggi).')
print(goodel)

In [ ]:
plt.close()
count=count.sort_values(by=['Count'],ascending=False)
count=count.drop([0])
count=count.reset_index(drop=True)
for i in range(0,len(count)):
    if count.at[i,'Count']>=limc:
        plt.barh(count.at[i,'Element'],width=count.at[i,'Count'],color='blue')
    else:
        plt.barh(count.at[i,'Element'],width=count.at[i,'Count'],color='grey')
plt.axvline(limc,color='red')

In [ ]:
mselected=[0,1,3,4,5,6,7,8,9,10,14,15]
selected=np.append(mselected,goodel)
selected=np.sort(selected)

In [ ]:
headers=[]
i=0
for i in range(0,len(selected)):
    headers.append(list(Excldata)[selected[i]])

Exclel=Excldata[headers]
Exclel.to_csv('./Settings/Data/DEFExclData.csv', index=False)

In [ ]:
###Si fa un confronto dei conteggi nei tre campioni
mscount=pd.read_csv('./Settings/Data/MSelcount.csv',names=['Element','CountMS'])
gbcount=pd.read_csv('./Settings/Data/GBelcount.csv',names=['Element','CountGB'])
exclcount=pd.read_csv('./Settings/Data/Exclelcount.csv',names=['Element','CountExcl'])
mscol=mscount[['CountMS']]
gbcol=gbcount[['CountGB']]
exclcol=exclcount[['CountExcl']]
compar=mscol.copy()
compar['CountGB']=gbcol
compar['CountExcl']=exclcol
compar=compar.set_index(mscount['Element'])
compar=compar.loc[(compar!=0).any(axis=1)]
plt.close()
compar.plot.bar(width=0.8,color=['#011627','#BAD7F2','#5DD39E'])
plt.ylabel('Counts')
plt.axhline(limms,color='#011627',linestyle=':',label='Threshold MS')
plt.axhline(limgb,color='#BAD7F2',linestyle=':',label='Threshold GB')
plt.axhline(limex,color='#5DD39E',linestyle=':',label='Threshold Excluded')
plt.legend(bbox_to_anchor=(1.005,0.5), loc="center left", borderaxespad=0)